In [1]:
import pandas as pd;
import numpy as np;
from sklearn.preprocessing import LabelEncoder;
from sklearn.tree import DecisionTreeClassifier;
from sklearn import tree;
from sklearn.metrics import f1_score;
import time

In [2]:
TRAINING_FILE = "bank_train.csv";
TEST_FILE = "bank_test.csv";
SAMPLE = "bank_sample_submission.csv"

In [3]:
df_train = pd.DataFrame.from_csv(TRAINING_FILE, parse_dates=True, index_col=None);
df_test = pd.DataFrame.from_csv(TEST_FILE, parse_dates=True, index_col=None);
df_sample = pd.DataFrame.from_csv(SAMPLE, parse_dates=True, index_col=None);

C:\Users\sajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
C:\Users\sajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
C:\Users\sajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#Remove rows from dataframe where job column has values unknown, unknown values are less in number 191 and 
#corresponding columns also contain -1 or unknown values for pdays and poutcome
df_train = df_train[df_train.job != 'unknown']
df_train = df_train[df_train.education != 'unknown']

In [5]:
job_enc = LabelEncoder()
df_train['job'] = job_enc.fit_transform(df_train['job'])

marital_enc = LabelEncoder()
df_train['marital'] = marital_enc.fit_transform(df_train['marital'])

edu_enc = LabelEncoder()
df_train['education'] = edu_enc.fit_transform(df_train['education'])

contact_enc = LabelEncoder()
df_train['contact'] = contact_enc.fit_transform(df_train['contact'])

month_enc = LabelEncoder()
df_train['month'] = month_enc.fit_transform(df_train['month'])

poutcome_enc = LabelEncoder()
df_train['poutcome'] = poutcome_enc.fit_transform(df_train['poutcome'])

df_train['defaulttransformed'] = pd.np.where(df_train['default'].astype(str)=="yes",1,0)
df_train['housingtransformed'] = pd.np.where(df_train['housing'].astype(str)=="yes",1,0)
df_train['loantransformed'] = pd.np.where(df_train['loan'].astype(str)=="yes",1,0)


job_enc2 = LabelEncoder()
df_test['job'] = job_enc2.fit_transform(df_test['job'])

marital_enc2 = LabelEncoder()
df_test['marital'] = marital_enc2.fit_transform(df_test['marital'])

edu_enc2 = LabelEncoder()
df_test['education'] = edu_enc2.fit_transform(df_test['education'])

contact_enc2 = LabelEncoder()
df_test['contact'] = contact_enc2.fit_transform(df_test['contact'])

month_enc2 = LabelEncoder()
df_test['month'] = month_enc2.fit_transform(df_test['month'])

poutcome_enc2 = LabelEncoder()
df_test['poutcome'] = poutcome_enc2.fit_transform(df_test['poutcome'])

df_test['defaulttransformed'] = pd.np.where(df_test['default'].astype(str)=="yes",1,0)
df_test['housingtransformed'] = pd.np.where(df_test['housing'].astype(str)=="yes",1,0)
df_test['loantransformed'] = pd.np.where(df_test['loan'].astype(str)=="yes",1,0)

In [6]:
def ageParser(age):
    if((age >= 18) and (age < 30)):
        value = 2
    elif((age >=30) and (age < 40)):
        value = 3
    elif((age >= 40) and (age < 50)):
        value = 4
    elif((age >=50) and (age < 60)):
        value = 5
    elif((age >=60) and (age < 70)):
        value = 6
    elif((age >= 70) and (age < 80)):
        value = 7
    elif((age >= 80) and (age < 90)):
        value = 8
    elif((age >= 90) and (age < 100)):
        value = 9
    else:
        value = 0
    return value

In [7]:
def campaignParser(num):
    if((num >= 0) and (num < 10)):
        value = 0
    elif((num >=10) and (num < 20)):
        value = 1
    elif((num >=20) and (num < 30)):
        value = 2
    elif((num >=30) and (num < 40)):
        value = 3
    elif((num >=40) and (num < 50)):
        value = 4
    elif((num >=50) and (num < 60)):
        value = 5
    elif((num >=60) and (num < 70)):
        value = 6
    else:
        value = 0
    return value

In [8]:
#df_train['age']= df_train.apply(lambda x: ageParser(x['age']), axis=1)
#df_test['age']= df_test.apply(lambda x: ageParser(x['age']), axis=1)

#df_train['campaign']= df_train.apply(lambda x: campaignParser(x['campaign']), axis=1)
#df_test['campaign']= df_test.apply(lambda x: campaignParser(x['campaign']), axis=1)

In [9]:
df_train = df_train.drop(['default','housing','loan','pdays','poutcome'],axis=1)
df_test = df_test.drop(['default','housing','loan','pdays','poutcome'],axis=1)

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27651 entries, 0 to 28933
Data columns (total 15 columns):
age                   27651 non-null int64
job                   27651 non-null int64
marital               27651 non-null int64
education             27651 non-null int64
balance               27651 non-null int64
contact               27651 non-null int64
day                   27651 non-null int64
month                 27651 non-null int64
duration              27651 non-null int64
campaign              27651 non-null int64
previous              27651 non-null int64
y                     27651 non-null int64
defaulttransformed    27651 non-null int32
housingtransformed    27651 non-null int32
loantransformed       27651 non-null int32
dtypes: int32(3), int64(12)
memory usage: 3.1 MB


In [11]:
attr_cols = df_train.columns

In [12]:
attr_cols

Index(['age', 'job', 'marital', 'education', 'balance', 'contact', 'day',
       'month', 'duration', 'campaign', 'previous', 'y', 'defaulttransformed',
       'housingtransformed', 'loantransformed'],
      dtype='object')

In [13]:
attr_cols = attr_cols.drop(['y'])

In [14]:
df_test_copy = df_test.copy()
df_test_copy = df_test_copy.drop(attr_cols, axis=1)

In [15]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(df_train[attr_cols], df_train['y'])
df_test_copy['y'] = clf.predict(df_test[attr_cols])

In [16]:
f1_score(df_sample['y'],df_test_copy['y'],average="micro")

0.49073818081282833

In [17]:
df_test_copy.to_csv('sample_bank_submission.csv', index=False)